In [50]:
import numpy as np
from numpy.linalg import inv as inv
from matplotlib import pyplot as plt
import torch

In [51]:
ipoints=500

C1=175e-15
C2=175e-15
C12=125e-15

R1=150
R2=150
R12=150

Ic1=250e-9
Ic2=250e-9
Ic12=350e-9

hbar=6.62e-34
e=1.6e-19

In [52]:
G1=1/R1
G2=1/R2
G12=1/R12
Gmat=np.array([[G1+G12, -G12],[-G12,G2+G12]])
Cmat=inv(np.array([[C1+C12,-C12],[-C12,C2+C12]]))
ehbar=2.0*e/hbar

In [54]:
iiIn=torch.linspace(-1e-6,1e-6,ipoints)
ijIn=torch.linspace(-1e-6,1e-6,ipoints)

In [55]:
A=torch.zeros(1,1,4,4,dtype=torch.float64)
A[0,0,0:2,2:4]=torch.eye(2)
A[0,0,2:4,2:4]=-torch.from_numpy(Cmat*Gmat)

B=torch.zeros(1,1,4,4,dtype=torch.float64)
B[0,0,2,0]=Ic1
B[0,0,3,1]=Ic2

Iin=torch.zeros(ipoints,ipoints,4,1,dtype=torch.float64)
for i in range(ipoints):
    for j in range(ipoints):
        Iin[i,j,2,0]=iiIn[i]
        Iin[i,j,3,0]=ijIn[j]

C=torch.zeros(1,1,4,4,dtype=torch.float64)
C[0,0,2:4,2:4]=torch.from_numpy(Cmat)

D2=torch.zeros(1,1,4,4,dtype=torch.float64)
D2[0,0,2:4,0:2]=torch.eye(2)
D2[0,0,2,1]=-1
D2[0,0,3,0]=-1

In [56]:
def psi_dot(psi_in):
    temp1=-ehbar*(torch.matmul(C,(torch.matmul(B,torch.sin(psi_in))+Ic12*torch.sin(torch.matmul(D2,psi_in))-Iin)))
    return torch.matmul(A,psi_in)+temp1

In [61]:
dt=1e-11
nsteps=200
avsteps=4000

In [62]:
def eval_psi(psi_in):
    return psi_in+psi_dot(psi_in)*dt

In [63]:
psi_int=torch.zeros(ipoints,ipoints,4,1,dtype=torch.float64)
psi_t=torch.zeros(ipoints,ipoints,4,1,dtype=torch.float32)

for i in range(nsteps):
    psi_int=eval_psi(psi_int)

for i in range(avsteps):
    psi_int=eval_psi(psi_int)
    psi_t+=psi_int

In [64]:
f=open('IV_Char_torch.txt','w')
f.close()
iiIn=np.linspace(-1e-6,1e-6,ipoints)
ijIn=np.linspace(-1e-6,1e-6,ipoints)

for i in range(ipoints):
    for j in range(ipoints):
        v=psi_t[i,j,:].numpy()/(ehbar*avsteps)
        v1=v[2][0]
        v2=v[3][0]
        f=open('IV_Char_torch.txt','a')
        f.write("{} {} {} {}\n" .format(iiIn[i],ijIn[j], v1,  v2))
        f.close()